In [2]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [3]:
#载入数据集
mnist=input_data.read_data_sets("MNIST_data",one_hot=True)    #我们要将mnist的0-9的标签转化为“one-hot vectors”。具体见课件

#每个批次的大小100
batch_size=100                      #以矩阵形式放入100张照片进行训练
n_batch=mnist.train.num_examples//batch_size     #数据集训练数量整除批次大小，就是一共需要训练多少个批次

#定义命名空间
with tf.name_scope('input'):
    x=tf.placeholder(tf.float32,[None,784],name='x-input')    
    y=tf.placeholder(tf.float32,[None,10],name='y-input')
    

with tf.name_scope('layer'):
    with tf.name_scope('weights'):
        W =tf.Variable(tf.zeros([784,10]),name='W')
    with tf.name_scope('biases'):
        b=tf.Variable(tf.zeros([10]),name='b')
    with tf.name_scope('wx_plus_b'):
        wx_plus_b=tf.matmul(x,W)+b
    with tf.name_scope('softmax'):
        prediction=tf.nn.softmax(wx_plus_b)
#创建一个简单的神经网络
W=tf.Variable(tf.zeros([784,10]))
b=tf.Variable(tf.zeros([10]))
prediction=tf.nn.softmax(tf.matmul(x,W)+b)

#定义二次代价函数
# loss=tf.reduce_mean(tf.square(y-prediction))
#跟softmax搭配使用的交叉熵代价函数
loss=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y,logits=prediction))
#使用梯度下降法
train_step=tf.train.GradientDescentOptimizer(0.2).minimize(loss)

#初始化变量
init=tf.global_variables_initializer()

with tf.name_scope('accuracy'):
    with tf.name_scope('correct_prediction'):
        
        #结果存放在一个布尔型列表中
        #argmax返回一维张量中最大的值所在的位置
        #tf.argmax(prediction,1) 求prediction最大的位置，并返回这个位置的标签，这里1是数字1。本例中判定概率最大的图片属于哪个标签，并返回标签
        #argmax(y,l) 本例中y的值为0或1，哪个位置是1就会返回这个位置。(0 0 1 0) 1在第二个标签位置上，就会返回2.
        #本例比较判断出来(predictino)的标签，与真实的标签是否一样，一样的话，就返回True,并存放在correct_prediction里，否则返回False
        correct_prediction=tf.equal(tf.argmax(y,1),tf.argmax(prediction,1)) #tf.equal()比较两个变量是否相同，相同的返回True，不同返回False
    with tf.name_scope('accuracy'):
        #求准确率
        accuracy=tf.reduce_mean(tf.cast(correct_prediction,tf.float32))    #cast()函数是转换类型，这里将布尔型转换为浮点型。true--1.0; false--0.0

with tf.Session() as sess:
    sess.run(init)
    writer = tf.summary.FileWriter('logs/',sess.graph)#这里logs/就是在当前工作目录下logs文件里写入文件，没有的话，会新建logs。写入的是graph图的结构
    for epoch in range(1):                                     #这里只关心结构，所以只循环一次
        for batch in range(n_batch):                            #该循环将所有批次的图片拿来训练一次，按批次循环，每批次100张图片
            batch_xs,batch_ys=mnist.train.next_batch(batch_size)#获得一个批次数量的图片（100个），图片数据保存在batch_xs，图片标签存在batch_ys
            sess.run(train_step,feed_dict={x:batch_xs,y:batch_ys})
        acc=sess.run(accuracy,feed_dict={x:mnist.test.images,y:mnist.test.labels})            #喂进去测试集的图片和标签
        print('Iter'+str(epoch)+',Testing Accuracy'+str(acc))

Extracting MNIST_data\train-images-idx3-ubyte.gz
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz
Iter0,Testing Accuracy0.8244
